In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keybert import KeyBERT
import time
from tqdm.notebook import tqdm
from spacy import displacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
sds_text = pd.read_csv('../data/sds_text.csv')

In [ ]:
sds_text = sds_text.set_index('Unnamed: 0')

In [ ]:
sds_text.head(2)

In [ ]:
#nlp.components

## Generating tokens

In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop
             and not token.is_digit
             and not token.is_punct
             and not token.is_space]
    return tokens

In [ ]:

sds_text['tokens'] = sds_text['episode_split_text'].apply(preprocess_text)

In [ ]:
sds_text.head()

In [ ]:
sds_text.to_csv('../data/sds_tokens.csv')

In [ ]:
X = sds_text[['tokens']]
y = sds_text['context_episode']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

## Entity Detection

In [ ]:
def entity_detection(text):
    doc = nlp(text)
    #entity = [(i, i.label_, i.label) for i in doc.ents]
    entity = displacy.render(doc, style = 'ent')
    return entity

In [ ]:
sds_1 = sds_text[sds_text['episode_number'] == 1]

In [ ]:
sds_1['entity'] = sds_1['episode_split_text'].apply(entity_detection)

In [ ]:
sds_1

In [ ]:
sds_text['entity'] = sds_text['episode_split_text'].apply(entity_detection)

In [ ]:
doc

In [ ]:
displacy.render(doc, style = 'ent')